<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
import requests

# Falcon 9 Launch Wiki page URL
url = "https://en.wikipedia.org/wiki/Falcon_9"

# Perform the GET request
response = requests.get(url)

# Check the status code to ensure the request was successful
if response.status_code == 200:
    print("Request successful!")
    html_content = response.text  # This will store the HTML content of the page
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")

# Output the first 500 characters of the HTML (for preview)
print(html_content[:500])


Request successful!
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-


Create a `BeautifulSoup` object from the HTML `response`


In [7]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [8]:
import requests
from bs4 import BeautifulSoup

# Static URL for the Falcon 9 and Falcon Heavy launches Wiki page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform the GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Preview the parsed data by printing the page title
    print(f"Page Title: {soup.title.string}")
    
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")


Request successful!
Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [9]:
import requests
from bs4 import BeautifulSoup

# Static URL for the Falcon 9 and Falcon Heavy launches Wiki page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform the GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Print the page title to verify the BeautifulSoup object was created properly
    print(f"Page Title: {soup.title.string}")
    
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")


Request successful!
Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [11]:
import requests
from bs4 import BeautifulSoup

# Static URL for the Falcon 9 and Falcon Heavy launches Wiki page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform the GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all tables on the page
    tables = soup.find_all('table')
    
    # Check if any tables were found
    if len(tables) > 0:
        # Extract the first table
        first_table = tables[0]
        
        # Print the first table's HTML structure to inspect
        print("First table HTML structure:")
        print(first_table.prettify()[:1000])  # Print the first 1000 characters to avoid excessive output
    else:
        print("No tables found on the page.")
    
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")


Request successful!
First table HTML structure:
<table class="col-begin" role="presentation">
 <tbody>
  <tr>
   <td class="col-break">
    <div class="mw-heading mw-heading3">
     <h3 id="Rocket_configurations">
      Rocket configurations
     </h3>
    </div>
    <div class="chart noresize" style="padding-top:10px;margin-top:1em;max-width:420px;">
     <div style="position:relative;min-height:320px;min-width:420px;max-width:420px;">
      <div style="float:right;position:relative;min-height:240px;min-width:320px;max-width:320px;border-left:1px black solid;border-bottom:1px black solid;">
       <div style="position:absolute;left:3px;top:224px;height:15px;min-width:18px;max-width:18px;background-color:LightSteelBlue;-webkit-print-color-adjust:exact;border:1px solid LightSteelBlue;border-bottom:none;overflow:hidden;" title="[[Falcon 9 v1.0]]: 2">
       </div>
       <div style="position:absolute;left:55px;top:224px;height:15px;min-width:18px;max-width:18px;background-color:LightStee

In [12]:
import requests
from bs4 import BeautifulSoup

# Static URL for the Falcon 9 and Falcon Heavy launches Wiki page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform the GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all tables on the page
    tables = soup.find_all('table')
    
    # Check if any tables were found
    if len(tables) > 0:
        # Extract the first table
        first_table = tables[0]
        
        # Find the first row (tr), which might contain column names
        first_row = first_table.find('tr')
        
        # Extract and print the text content of each cell in the first row
        column_names = [cell.text.strip() for cell in first_row.find_all(['th', 'td'])]
        print("Column names found in the first table:")
        print(column_names)
    else:
        print("No tables found on the page.")
    
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")


Request successful!
Column names found in the first table:
["Rocket configurations\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n5\n\n10\n\n15\n\n20\n\n25\n\n30\n\n\n\n'10\n\n'11\n\n'12\n\n'13\n\n'14\n\n'15\n\n'16\n\n'17\n\n'18\n\n'19\n\n'20\n\n'21\n\n\n\n\n\xa0 Falcon 9 v1.0\n\xa0 Falcon 9 v1.1\n\xa0 Falcon 9 Full Thrust\n\xa0 Falcon 9 FT (reused)\n\xa0 Falcon 9 Block 5\n\xa0 Falcon 9 B5 (reused)\n\xa0 Falcon Heavy", "Launch sites\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n5\n\n10\n\n15\n\n20\n\n25\n\n30\n\n\n\n'10\n\n'11\n\n'12\n\n'13\n\n'14\n\n'15\n\n'16\n\n'17\n\n'18\n\n'19\n\n'20\n\n'21\n\n\n\n\n\xa0 CCSFS, SLC-40\n\xa0 KSC, LC-39A\n\xa0 VAFB, SLC-4E"]


Starting from the third table is our target table contains the actual launch records.


In [13]:
import requests
from bs4 import BeautifulSoup

# Static URL for the Falcon 9 and Falcon Heavy launches Wiki page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform the GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all tables on the page
    tables = soup.find_all('table')
    
    # Ensure there are at least three tables
    if len(tables) >= 3:
        # Extract the third table (index starts at 0)
        third_table = tables[2]
        
        # Print the third table's HTML structure for inspection
        print("Third table HTML structure:")
        print(third_table.prettify()[:1000])  # Print the first 1000 characters for a preview
    else:
        print("Less than 3 tables found on the page.")
    
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")


Request successful!
Third table HTML structure:
<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     Booster
    </a>
    <sup class="reference" id="cite_ref-booster_11-0">
     <a href="#cite_note-booster-11">
      <span class="cite-bracket">
       [
      </span>
      b
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_12-0">
     <a href="#cite_note-Dragon-12">
      <span class="cite-bracket">
       [
      </span>
     

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [15]:
def extract_column_from_header(header_element):
    # Extracts the text from the header element, removing extra whitespace
    return header_element.text.strip()

In [17]:
import requests
from bs4 import BeautifulSoup

# Static URL for the Falcon 9 and Falcon Heavy launches Wiki page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Function to extract column names from <th> elements
def extract_column_from_header(header_element):
    # Extract the column name text and strip any extra spaces
    return header_element.text.strip()

# Perform the GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all tables on the page
    tables = soup.find_all('table')
    
    # Ensure there are at least three tables
    if len(tables) >= 3:
        # Extract the third table (index 2)
        third_table = tables[2]
        
        # Find all <th> elements in the third table (table headers)
        headers = third_table.find_all('th')
        
        # Extract column names from each header element
        column_names = [extract_column_from_header(header) for header in headers]
        
        # Print the extracted column names
        print("Column names found in the third table:")
        print(column_names)
    else:
        print("Less than 3 tables found on the page.")
    
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")


Request successful!
Column names found in the third table:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']


Check the extracted column names


In [18]:
import requests
from bs4 import BeautifulSoup

# Static URL for the Falcon 9 and Falcon Heavy launches Wiki page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Function to extract column names from <th> elements
def extract_column_from_header(header_element):
    # Extract the column name text and strip any extra spaces
    return header_element.text.strip()

# Perform the GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all tables on the page
    tables = soup.find_all('table')
    
    # Ensure there are at least three tables
    if len(tables) >= 3:
        # Extract the third table (index 2)
        third_table = tables[2]
        
        # Find all <th> elements in the third table (table headers)
        headers = third_table.find_all('th')
        
        # Extract column names from each header element
        column_names = [extract_column_from_header(header) for header in headers]
        
        # Print the extracted column names
        print("Column names found in the third table:")
        print(column_names)
    else:
        print("Less than 3 tables found on the page.")
    
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")


Request successful!
Column names found in the third table:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [19]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Static URL for the Falcon 9 and Falcon Heavy launches Wiki page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Function to extract column names from <th> elements
def extract_column_from_header(header_element):
    # Extract the column name text and strip any extra spaces
    return header_element.text.strip()

# Perform the GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all tables on the page
    tables = soup.find_all('table')
    
    # Ensure there are at least three tables
    if len(tables) >= 3:
        # Extract the third table (index 2)
        third_table = tables[2]
        
        # Find all <th> elements in the third table (table headers)
        headers = third_table.find_all('th')
        
        # Extract column names from each header element
        column_names = [extract_column_from_header(header) for header in headers]
        print("Column names found in the third table:")
        print(column_names)
        
        # Create an empty dictionary to hold the table data
        table_data = {col: [] for col in column_names}
        
        # Find all rows in the table (skip the first one as it contains headers)
        rows = third_table.find_all('tr')[1:]  # Skipping the header row
        
        # Loop through each row and extract cell data
        for row in rows:
            cells = row.find_all('td')
            
            # Check if the number of cells matches the number of headers
            if len(cells) == len(column_names):
                for i, cell in enumerate(cells):
                    table_data[column_names[i]].append(cell.text.strip())
            else:
                print(f"Skipping row due to mismatch: {len(cells)} cells found, expected {len(column_names)}")
        
        # Convert the dictionary into a Pandas DataFrame
        df = pd.DataFrame(table_data)
        
        # Display the DataFrame
        print("Data extracted into the DataFrame:")
        print(df.head())  # Display the first 5 rows of the DataFrame
    else:
        print("Less than 3 tables found on the page.")
    
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")


Request successful!
Column names found in the third table:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 5 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skip

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Static URL for the Falcon 9 and Falcon Heavy launches Wiki page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Function to extract column names from <th> elements
def extract_column_from_header(header_element):
    # Extract the column name text and strip any extra spaces
    return header_element.text.strip()

# Perform the GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all tables on the page
    tables = soup.find_all('table')
    
    # Ensure there are at least three tables
    if len(tables) >= 3:
        # Extract the third table (index 2)
        third_table = tables[2]
        
        # Find all <th> elements in the third table (table headers)
        headers = third_table.find_all('th')
        
        # Extract column names from each header element
        column_names = [extract_column_from_header(header) for header in headers]
        print("Column names found in the third table:")
        print(column_names)
        
        # Initialize the launch_dict with empty lists for each column
        launch_dict = {col: [] for col in column_names}
        
        # Find all rows in the table (skip the first one as it contains headers)
        rows = third_table.find_all('tr')[1:]  # Skipping the header row
        
        # Loop through each row and extract cell data
        for row in rows:
            cells = row.find_all('td')
            
            # Check if the number of cells matches the number of headers
            if len(cells) == len(column_names):
                for i, cell in enumerate(cells):
                    launch_dict[column_names[i]].append(cell.text.strip())  # Append cell data to the corresponding column
            else:
                print(f"Skipping row due to mismatch: {len(cells)} cells found, expected {len(column_names)}")
        
        # Convert the launch_dict into a Pandas DataFrame
        df = pd.DataFrame(launch_dict)
        
        # Display the DataFrame
        print("Data extracted into the DataFrame:")
        print(df.head())  # Display the first 5 rows of the DataFrame
    else:
        print("Less than 3 tables found on the page.")
    
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")


Request successful!
Column names found in the third table:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 5 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skip

df = pd.DataFrame({ key: pd.Series(value) for key, value in launch_dict.items() })
After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Static URL for the Falcon 9 and Falcon Heavy launches Wiki page
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Function to extract column names from <th> elements
def extract_column_from_header(header_element):
    # Extract the column name text and strip any extra spaces
    return header_element.text.strip()

# Perform the GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find all tables on the page
    tables = soup.find_all('table')
    
    # Ensure there are at least three tables
    if len(tables) >= 3:
        # Extract the third table (index 2)
        third_table = tables[2]
        
        # Find all <th> elements in the third table (table headers)
        headers = third_table.find_all('th')
        
        # Extract column names from each header element
        column_names = [extract_column_from_header(header) for header in headers]
        print("Column names found in the third table:")
        print(column_names)
        
        # Initialize the launch_dict with empty lists for each column
        launch_dict = {col: [] for col in column_names}
        
        # Find all rows in the table (skip the first one as it contains headers)
        rows = third_table.find_all('tr')[1:]  # Skipping the header row
        
        # Loop through each row and extract cell data
        for row in rows:
            cells = row.find_all('td')
            
            # Check if the number of cells matches the number of headers
            if len(cells) == len(column_names):
                for i, cell in enumerate(cells):
                    launch_dict[column_names[i]].append(cell.text.strip())  # Append cell data to the corresponding column
            else:
                print(f"Skipping row due to mismatch: {len(cells)} cells found, expected {len(column_names)}")
        
        # Specify dtype for columns where empty lists might exist
        dtype_dict = {
            'Flight No.': 'int64',  # Assuming Flight No. should be integer
            'Date andtime (UTC)': 'str',
            'Version,Booster [b]': 'str',
            'Launch site': 'str',
            'Payload[c]': 'str',
            'Payload mass': 'str',  # Payload mass can be a string if there are units like kg
            'Orbit': 'str',
            'Customer': 'str',
            'Launchoutcome': 'str',
            'Boosterlanding': 'str',
            # Add dtypes for columns '1' through '7' or others if needed
        }
        
        # Convert the launch_dict into a Pandas DataFrame, with dtype specified for empty lists
        df = pd.DataFrame(launch_dict).astype(dtype_dict, errors='ignore')
        
        # Display the DataFrame
        print("Data extracted into the DataFrame with specified dtype:")
        print(df.dtypes)  # Display the data types of each column
        print(df.head())  # Display the first 5 rows of the DataFrame
        
    else:
        print("Less than 3 tables found on the page.")
    
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")


Request successful!
Column names found in the third table:
['Flight No.', 'Date andtime (UTC)', 'Version,Booster [b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 5 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skipping row due to mismatch: 9 cells found, expected 17
Skipping row due to mismatch: 1 cells found, expected 17
Skip

In [25]:
import requests
import pandas as pd

# Step 1: Perform an HTTP GET request to the SpaceX API (for example, the launches endpoint)
spacex_api_url = "https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_api_url)

# Step 2: Convert the response to JSON and normalize it into a DataFrame
launch_data = response.json()
df = pd.json_normalize(launch_data)

# Step 3: Extract the first row from the 'static_fire_date_utc' column and get the year
first_static_fire_date = df['static_fire_date_utc'].iloc[0]
year = first_static_fire_date[:4]  # Extract the first four characters (year)
print(f"The year in the first row of 'static_fire_date_utc' is: {year}")


The year in the first row of 'static_fire_date_utc' is: 2006


In [26]:
import requests
import pandas as pd

# Step 1: Perform an HTTP GET request to the SpaceX API (for launches)
spacex_api_url = "https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_api_url)

# Step 2: Convert the response to JSON and normalize it into a DataFrame
launch_data = response.json()
launch_df = pd.json_normalize(launch_data)

# Step 3: Perform an HTTP GET request to get rocket details
rocket_api_url = "https://api.spacexdata.com/v4/rockets"
rocket_response = requests.get(rocket_api_url)
rocket_data = rocket_response.json()
rocket_df = pd.json_normalize(rocket_data)

# Step 4: Print the columns of the rocket DataFrame to identify the column that contains the rocket name
print("Rocket DataFrame columns:")
print(rocket_df.columns)

# Step 5: Merge the launch data with the rocket data using the correct ID from the 'rocket' column
merged_df = launch_df.merge(rocket_df[['id', 'name']], left_on='rocket', right_on='id', how='left')

# Step 6: Print the columns of the merged DataFrame to ensure the merge worked correctly
print("Merged DataFrame columns:")
print(merged_df.columns)

# Step 7: Now filter the DataFrame to keep only Falcon 9 launches based on the correct column for the rocket name
# Verify that the 'name' column exists in the merged_df
if 'name' in merged_df.columns:
    falcon_9_df = merged_df[merged_df['name'] == 'Falcon 9']
    # Step 8: Count the number of Falcon 9 launches
    falcon_9_count = len(falcon_9_df)
    print(f"The number of Falcon 9 launches is: {falcon_9_count}")




Rocket DataFrame columns:
Index(['payload_weights', 'flickr_images', 'name', 'type', 'active', 'stages',
       'boosters', 'cost_per_launch', 'success_rate_pct', 'first_flight',
       'country', 'company', 'wikipedia', 'description', 'id', 'height.meters',
       'height.feet', 'diameter.meters', 'diameter.feet', 'mass.kg', 'mass.lb',
       'first_stage.thrust_sea_level.kN', 'first_stage.thrust_sea_level.lbf',
       'first_stage.thrust_vacuum.kN', 'first_stage.thrust_vacuum.lbf',
       'first_stage.reusable', 'first_stage.engines',
       'first_stage.fuel_amount_tons', 'first_stage.burn_time_sec',
       'second_stage.thrust.kN', 'second_stage.thrust.lbf',
       'second_stage.payloads.composite_fairing.height.meters',
       'second_stage.payloads.composite_fairing.height.feet',
       'second_stage.payloads.composite_fairing.diameter.meters',
       'second_stage.payloads.composite_fairing.diameter.feet',
       'second_stage.payloads.option_1', 'second_stage.reusable',
       

In [27]:
import requests
import pandas as pd

# Step 1: Perform an HTTP GET request to the SpaceX API (for launches)
spacex_api_url = "https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_api_url)

# Step 2: Convert the response to JSON and normalize it into a DataFrame
launch_data = response.json()
df = pd.json_normalize(launch_data)

# Step 3: Check the columns in the DataFrame to find the correct column name
print(df.columns)

# Step 4: If 'landingPad' is present with a different name, adjust it. Assuming the column might be 'launchpad'
# For now, let's count missing values in all columns related to landing/launch pads
possible_columns = [col for col in df.columns if 'pad' in col.lower()]
print(f"Possible columns related to pads: {possible_columns}")

# If 'launchpad' is identified, count missing values (you can replace 'launchpad' with the correct column name)
if 'launchpad' in df.columns:
    missing_values_count = df['launchpad'].isnull().sum()
    print(f"The number of missing values in the 'launchpad' column is: {missing_values_count}")
else:
    print("The 'landingPad' or similar column does not exist in the DataFrame.")


Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Possible columns related to pads: ['launchpad']
The number of missing values in the 'launchpad' column is: 0


In [28]:
import requests
import pandas as pd

# Step 1: Perform an HTTP GET request to the SpaceX API (for launches)
spacex_api_url = "https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_api_url)

# Step 2: Convert the response to JSON and normalize it into a DataFrame
launch_data = response.json()
df = pd.json_normalize(launch_data)

# Step 3: Check the available columns in the DataFrame to identify the correct column related to 'landingPad'
print(df.columns)

# Step 4: Assuming the column might be 'landing_pad' or something similar, let's count the missing values
if 'landing_pad' in df.columns:
    missing_values_count = df['landing_pad'].isnull().sum()
    print(f"The number of missing values in the 'landing_pad' column is: {missing_values_count}")
else:
    print("The 'landing_pad' column does not exist in the DataFrame.")

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
The 'landing_pad' column does not exist in the DataFrame.


In [30]:
import requests
import pandas as pd

# Step 1: Perform an HTTP GET request to the SpaceX API (for launches)
spacex_api_url = "https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_api_url)

# Step 2: Convert the response to JSON and normalize it into a DataFrame
launch_data = response.json()
df = pd.json_normalize(launch_data)

# Step 3: Check for missing values in the 'launchpad' column
missing_values_count = df['launchpad'].isnull().sum()

# Step 4: Output the number of missing values in the 'launchpad' column
print(f"The number of missing values in the 'launchpad' column is: {missing_values_count}")


The number of missing values in the 'launchpad' column is: 0


In [31]:
# Check for missing values in the 'launchpad' column, including empty strings and placeholders
missing_values_count = df['launchpad'].apply(lambda x: x is None or x == "" or x == "N/A").sum()

# Output the number of missing values in the 'launchpad' column
print(f"The number of missing values in the 'launchpad' column is: {missing_values_count}")


The number of missing values in the 'launchpad' column is: 0


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


In [32]:
# Check for missing values in the 'launchpad' column, including more placeholders
missing_values_count = df['launchpad'].apply(lambda x: x is None or x == "" or x == "N/A" or x == "unknown" or x == "None").sum()

# Output the number of missing values in the 'launchpad' column
print(f"The number of missing values in the 'launchpad' column is: {missing_values_count}")


The number of missing values in the 'launchpad' column is: 0


In [33]:
# Print the exact column names to confirm the correct name of the 'launchpad' column
print(df.columns)


Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')


In [34]:
# Output all unique values in the 'launchpad' column
unique_values = df['launchpad'].unique()
print(unique_values)


['5e9e4502f5090995de566f86' '5e9e4501f509094ba4566f84'
 '5e9e4502f509092b78566f87' '5e9e4502f509094188566f88']


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [35]:
import requests
import pandas as pd

# Step 1: Perform an HTTP GET request to the SpaceX API (for launches)
spacex_api_url = "https://api.spacexdata.com/v4/launches"
response = requests.get(spacex_api_url)

# Step 2: Convert the response to JSON and normalize it into a DataFrame
launch_data = response.json()
df = pd.json_normalize(launch_data)

# Step 3: Check the available columns to ensure 'launchpad' exists
print("Available columns in DataFrame:")
print(df.columns)

# Step 4: Check for missing values in the 'launchpad' column
if 'launchpad' in df.columns:
    missing_values_count = df['launchpad'].isnull().sum()
    print(f"Number of missing values in 'launchpad': {missing_values_count}")

    # Step 5: Check for common placeholders for missing values ('N/A', 'unknown', 'None', etc.)
    placeholders = ['N/A', 'None', 'unknown', '']
    placeholder_count = df['launchpad'].isin(placeholders).sum()
    print(f"Number of placeholder values in 'launchpad': {placeholder_count}")

    # Step 6: Check the data type of the 'launchpad' column
    print(f"Data type of 'launchpad': {df['launchpad'].dtype}")

    # Step 7: Output unique values in the 'launchpad' column
    print("Unique values in 'launchpad' column:")
    print(df['launchpad'].unique())
else:
    print("The 'launchpad' column does not exist in the DataFrame.")


Available columns in DataFrame:
Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Number of missing values in 'launchpad': 0
Number of placeholder values in 'launchpad': 0
Data type of 'launchpad': object
Unique values in 'launchpad' column:
['5e9e4502f509099

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
